# NDVI Varience NOtebook
These are the steps to follow
Step 2A — NDVI + Gradient Extraction

🔥 Step 2B — Adaptive NDVI-guided SLIC

I implement improved segmentation.
We test per-image segment count & boundaries.

🔥 Step 2C — Prompt extraction

We extract:

centroids

gradient boundary mask

NDVI interior points

🔥 Step 2D — SAM prediction & evaluation

Then we run SAM on these signals.

                 ┌──────────────────────────────────────────┐
                 │                Step 2A                   │
                 │     Load & Normalize Sentinel-2 Data     │
                 │  • NDVI (time-series → best timestamp)   │
                 │  • RGB bands                             │
                 │  • Ground-truth mask                     │
                 │  • Parcels for this tile                 │
                 └──────────────────────────────────────────┘
                                   │
                                   ▼
                 ┌──────────────────────────────────────────┐
                 │                Step 2B                   │
                 │       NDVI Variability Analysis          │
                 │  • NDVI mean, variance, gradient maps    │
                 │  • NDVI edge map (field boundaries)      │
                 │  • threshold maps (high/low NDVI zones)  │
                 └──────────────────────────────────────────┘
                                   │
                                   ▼
                 ┌──────────────────────────────────────────┐
                 │                Step 2C                   │
                 │      NDVI-Guided Superpixel Segments     │
                 │  (NOT constant SLIC!)                    │
                 │  • dynamic segments based on NDVI var    │
                 │  • more clusters where NDVI unstable     │
                 │  • fewer clusters where NDVI uniform     │
                 └──────────────────────────────────────────┘
                                   │
                                   ▼
                 ┌──────────────────────────────────────────┐
                 │                Step 2D                   │
                 │      NDVI-Guided Prompt Generation       │
                 │  • centroids of superpixels              │
                 │  • boundary pixels from NDVI edges       │
                 │  • NDVI cluster centers (k-means)        │
                 └──────────────────────────────────────────┘
                                   │
                                   ▼
                 ┌──────────────────────────────────────────┐
                 │                Step 2E                   │
                 │   SAM Inference + IoU/F1 Evaluation      │
                 │  • Prompt-to-Mask generation             │
                 │  • Compare with GT mask                  │
                 │  • Export CSV results                    │
                 └──────────────────────────────────────────┘


In [1]:
import os
import sys
if '..' not in sys.path:
    sys.path.append('..')
from src.step2_ndvivariance import load_all_data

from shapely.geometry import mapping
import numpy as np
import rasterio
from rasterio.features import rasterize
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from src.config import *

In [2]:
#2A NDVI Gradient Extract

dataset = load_all_data()

print("\n=== Step 2A Test Summary ===")

for entry in dataset:
    print(f"\nImage ID: {entry['id']}")
    print(" Timesteps:", entry["n_timesteps"])
    print(" NDVI mean range:", float(entry["ndvi_mean"].min()), "→", float(entry["ndvi_mean"].max()))
    print(" NDVI var range :", float(entry["ndvi_var"].min()),  "→", float(entry["ndvi_var"].max()))
    print(" Mask shape:", entry["mask"].shape)
    print(" Num parcels:", len(entry["parcels"]))

[Step-0] Image 1022: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0545
[Step-0] Image 1023: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0160
[Step-0] Image 1024: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0439
[Step-0] Image 1027: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0587
[Step-0] Image 1028: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0463
[Step-0] Image 1029: NDVI time-series loaded (T=6) | NDVI variance threshold=0.1050
[Step-0] Image 1031: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0850
[Step-0] Image 1032: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0618
[Step-0] Image 1099: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0321
[Step-0] Image 1100: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0355
[Step-0] Image 1101: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0923
[Step-0] Image 1103: NDVI time-series loaded (T=6) | NDVI variance threshold

In [3]:
#2B NDVI Variance Mask Test
from src.step2_ndvivariance import compute_ndvi_variance_mask


print("=== Step 2B: NDVI Variance Mask ===")

for item in dataset:

    img_id = item["id"]
    ndvi_var = item["ndvi_var"]

    mean = float(ndvi_var.mean())
    std  = float(ndvi_var.std())
    threshold = mean + 0.5 * std  # adaptive threshold

    ndvi_mask = (ndvi_var > threshold).astype(np.uint8)

    # --- KEY LINE: STORE THE RESULT ---
    item["ndvi_mask"] = ndvi_mask

    print(f"\nImage {img_id}")
    print(f" Mean = {mean:.4f}, Std = {std:.4f}, Threshold = {threshold:.4f}")
    print(f" Active pixels: {ndvi_mask.sum()} out of {ndvi_mask.size}")


=== Step 2B: NDVI Variance Mask ===

Image 1022
 Mean = 0.0293, Std = 0.0252, Threshold = 0.0419
 Active pixels: 19811 out of 65536

Image 1023
 Mean = 0.0111, Std = 0.0049, Threshold = 0.0136
 Active pixels: 17321 out of 65536

Image 1024
 Mean = 0.0193, Std = 0.0246, Threshold = 0.0316
 Active pixels: 10185 out of 65536

Image 1027
 Mean = 0.0272, Std = 0.0314, Threshold = 0.0430
 Active pixels: 14379 out of 65536

Image 1028
 Mean = 0.0216, Std = 0.0247, Threshold = 0.0339
 Active pixels: 13351 out of 65536

Image 1029
 Mean = 0.0381, Std = 0.0669, Threshold = 0.0716
 Active pixels: 9942 out of 65536

Image 1031
 Mean = 0.0474, Std = 0.0375, Threshold = 0.0662
 Active pixels: 22748 out of 65536

Image 1032
 Mean = 0.0343, Std = 0.0275, Threshold = 0.0480
 Active pixels: 15821 out of 65536

Image 1099
 Mean = 0.0136, Std = 0.0185, Threshold = 0.0228
 Active pixels: 16161 out of 65536

Image 1100
 Mean = 0.0191, Std = 0.0164, Threshold = 0.0273
 Active pixels: 13171 out of 65536

Imag

In [4]:
from src.Archive_step2_prompts_superpixels import run_superpixel_generation

superpixel_results = run_superpixel_generation(dataset)
superpixel_results



[2D] Generating superpixel prompts for image 1022...
[2D] Image 1022: 116 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_1022.npy

[2D] Generating superpixel prompts for image 1023...
[2D] Image 1023: 49 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_1023.npy

[2D] Generating superpixel prompts for image 1024...
[2D] Image 1024: 93 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_1024.npy

[2D] Generating superpixel prompts for image 1027...
[2D] Image 1027: 108 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_1027.npy

[2D] Generating superpixel prompts for image 1028...
[2D] Image 1028: 103 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_1028.npy

[2D] Generating superpixel prompts for image 1029...
[2D] Image 1029: 119 superpixel prompts saved → d:\project_alpha\data\results\prompts\superpixel_prompts_102

{1022: {'centroids': array([[  8.49606299,   8.82677165],
         [ 72.29483283,   9.86930091],
         [125.01546392,  16.3814433 ],
         [ 20.85920578,  26.19494585],
         [ 65.16742081,  40.31447964],
         [ 22.6125    ,  39.34375   ],
         [ 13.66144201,  48.20689655],
         [ 31.01466276,  54.5483871 ],
         [ 56.07692308,  66.64190981],
         [ 60.55555556,  53.68376068],
         [ 45.5942623 ,  62.71311475],
         [ 34.21010638,  65.87765957],
         [111.66203704,  72.92592593],
         [ 22.18726592,  75.67041199],
         [ 42.83076923,  74.51538462],
         [  8.45604396,  80.54395604],
         [ 53.85526316,  82.13596491],
         [117.15879828,  83.34334764],
         [ 34.43277311,  86.23529412],
         [ 22.04605263,  88.71052632],
         [ 75.97087379,  90.14239482],
         [ 69.55208333,  94.28645833],
         [  6.24113475,  95.24822695],
         [ 49.59166667,  94.46666667],
         [122.376     ,  93.904     ],
      

Loading SAM

In [5]:
from src.sam_tools import load_sam_model



# Config - adjust if your paths differ
PROMPTS_DIR = os.path.join("data", "results", "prompts")
PRED_MASK_DIR = os.path.join("data", "results", "predicted_masks")
PRED_VIS_DIR = os.path.join("data", "results", "pred_vis")
SUMMARY_CSV = os.path.join("data", "results", "sam_summary.csv")
os.makedirs(PRED_MASK_DIR, exist_ok=True)
os.makedirs(PRED_VIS_DIR, exist_ok=True)


def compute_iou(gt, pred):
    # expects boolean or 0/1 arrays same shape
    gtb = (gt > 0).astype(np.uint8)
    pdb = (pred > 0).astype(np.uint8)
    inter = (gtb & pdb).sum()
    union = (gtb | pdb).sum()
    if union == 0:
        return 1.0 if inter == 0 else 0.0
    return inter / union

def binarize_mask(mask):
    return (mask > 0.5).astype(np.uint8)


In [6]:
# C Helper to call SAM predictor for one image

import os
import sys
if '..' not in sys.path:
    sys.path.append('..')
from src.config import PROMPTS_DIR
from src.step2_scaling import load_nc_image

predictor = load_sam_model()

def predict_from_points(predictor, rgb_image, points_xy):
    """
    predictor: SAM predictor object already set on the image (see your earlier code pattern)
    rgb_image: HxWx3 float (0..1) or uint8; predictor typically requires the RGB image to be set via predictor.set_image(image)
    points_xy: np.array of shape (N,2) in pixel coords (x, y)
    Returns: predicted_mask (binary HxW) selected using best-IoU among multimask outputs
    """
    # SAM expects point coords in shape (N,2) as float32 and point labels (all 1)
    if points_xy is None or len(points_xy) == 0:
        return np.zeros(rgb_image.shape[:2], dtype=np.uint8)
    point_coords = points_xy.astype(np.float32)
    point_labels = np.ones(len(point_coords), dtype=np.int32)

    # Ensure predictor has the image set
    predictor.set_image((rgb_image * 255).astype(np.uint8) if rgb_image.max() <= 1.1 else rgb_image.astype(np.uint8))

    # Call predictor (adjust args if your SAM wrapper differs)
    masks, scores, logits = predictor.predict(
        point_coords=point_coords,
        point_labels=point_labels,
        multimask_output=True
    )
    # masks: list or array of shape (K, H, W) where K is number of masks returned
    return masks, scores

# MAIN LOOP
rows = []
for item in dataset:
    image_id = item["id"]
    print(f"\n[RUN] Image {image_id}")
    #rgb = item.get("rgb")              # shape H,W,3 (float 0..1) from your loader
    ndvi_tmp, rgb = load_nc_image(item["nc_path"])
    gt_mask = item.get("mask")         # shape H,W (0/1)
    h, w = gt_mask.shape

# Skip images with no foreground parcels
    if gt_mask.sum() == 0:
        print(f"[SKIP] Image {image_id}: GT mask empty (no parcels).")
        continue

# Skip images where NDVI activity is too low
    if item["ndvi_mask"].sum() < 200:
        print(f"[SKIP] Image {image_id}: NDVI mask too sparse.")
        continue

    # load saved centroids (N x 2: [x,y] or [col,row] depending on how you saved — adjust if necessary)
    prompts_file = os.path.join(PROMPTS_DIR, f"superpixel_prompts_{image_id}.npy")
    if not os.path.exists(prompts_file):
        print("  -> no prompts found, skipping")
        continue

    centroids = np.load(prompts_file)  # expected shape (N, 2) as (row, col) or (x,y) — adjust interpretation below
    # --- Ensure we convert to SAM expected (x, y) pixel coords ---
    # If you saved as (row, col) -> convert: points_xy = np.stack([cols, rows], axis=1)
    if centroids.shape[1] == 2:
        # heuristics: if values look like ints in range [0,h) and [0,w) but order (row,col),
        # check first element: if first coord <= h and second <= w -> treat as (row,col)
        r0, c0 = centroids[0]
        if r0 <= h and c0 <= w:
            points_xy = np.vstack([centroids[:,1], centroids[:,0]]).T  # (x=col, y=row)
        else:
            points_xy = centroids.copy()
    else:
        raise ValueError("unexpected centroids shape: " + str(centroids.shape))

    # If you want to limit points for predictor (SAM might have limits), sample top k
    MAX_POINTS = 200
    if len(points_xy) > MAX_POINTS:
        points_xy = points_xy[:MAX_POINTS]

    # run predictor
    try:
        masks, scores = predict_from_points(predictor, rgb, points_xy)
    except Exception as e:
        print("  -> predictor error:", e)
        continue

    # masks shape: (K, H, W) or list; compute IoU of each returned mask and pick best
    best_iou = -1.0
    best_mask = None
    best_score = None
    for k, m in enumerate(masks):
        # m may be boolean already
        mp = binarize_mask(m)
        iou = compute_iou(gt_mask, mp)
        if iou > best_iou:
            best_iou = iou
            best_mask = mp
            try:
                best_score = scores[k]
            except:
                best_score = None

    # store predicted mask
    np.save(os.path.join(PRED_MASK_DIR, f"pred_mask_{image_id}.npy"), best_mask.astype(np.uint8))
    # also write a small visualization: gt vs pred overlay
    fig, ax = plt.subplots(1,3, figsize=(12,4))
    ax[0].imshow(np.clip(rgb,0,1)); ax[0].set_title("RGB"); ax[0].axis("off")
    ax[1].imshow(gt_mask, cmap="gray"); ax[1].set_title("GT Mask"); ax[1].axis("off")
    ax[2].imshow(best_mask, cmap="gray"); ax[2].set_title(f"SAM Pred (IoU={best_iou:.3f})"); ax[2].axis("off")
    fig.tight_layout()
    fig.savefig(os.path.join(PRED_VIS_DIR, f"vis_{image_id}.png"), dpi=150)
    plt.close(fig)

    # Compute precision/recall/f1 at pixel-level
    gt_vec = (gt_mask > 0).astype(int).ravel()
    pred_vec = best_mask.ravel().astype(int)
    # handle edge cases: if both empty, set metrics to 1.0
    if gt_vec.sum() == 0 and pred_vec.sum() == 0:
        prec = rec = f1 = 1.0
    else:
        prec = precision_score(gt_vec, pred_vec, zero_division=0)
        rec = recall_score(gt_vec, pred_vec, zero_division=0)
        f1 = f1_score(gt_vec, pred_vec, zero_division=0)

    rows.append({
        "image_id": image_id,
        "n_prompts": len(points_xy),
        "best_iou": float(best_iou),
        "pred_score": float(best_score) if best_score is not None else None,
        "precision": float(prec),
        "recall": float(rec),
        "f1": float(f1)
    })
    print(f"  -> Image {image_id}: prompts={len(points_xy)}, IoU={best_iou:.3f}, F1={f1:.3f}")

# Save summary CSV
df = pd.DataFrame(rows).sort_values("image_id")
df.to_csv(SUMMARY_CSV, index=False)
print("\nSaved summary:", SUMMARY_CSV)
print(df)
# ======================================================================

D:\segment-anything\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)



[RUN] Image 1022
Available variables in NL_1022.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
  -> Image 1022: prompts=116, IoU=0.003, F1=0.005

[RUN] Image 1023
Available variables in NL_1023.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
[SKIP] Image 1023: GT mask empty (no parcels).

[RUN] Image 1024
Available variables in NL_1024.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
  -> Image 1024: prompts=93, IoU=0.060, F1=0.113

[RUN] Image 1027
Available variables in NL_1027.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
  -> Image 1027: prompts=109, IoU=0.213, F1=0.351

[RUN] Image 1028
Available variables in NL_1028.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
  -> Image 1028: prompts=103, IoU=0.210, F1=0.348

[RUN] Image 1029
Available variables in NL_1029.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
  -> Image 1029: prompts=119, IoU=0.018, F1=0.035

[RUN] Image 1031
Available variables in NL_1031.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref

In [3]:
# Step 2D-B: RRefined prompts

from src.step2_prompts_refine import refine_prompts_by_ndvi_variance

results = refine_prompts_by_ndvi_variance(dataset, top_percent=0.25)
results



========== Step 2B-REFINE: Reducing prompts per image using NDVI variance ==========

[REFINE] Image 1022
  NDVI cutoff=0.04762, active_pixels=16384
  -> Original=116, Refined=30, Dropped=86
[SKIP] Image 1023: GT empty — no refinement needed.
[REFINE] Image 1024
  NDVI cutoff=0.01970, active_pixels=16384
  -> Original=93, Refined=21, Dropped=72
[REFINE] Image 1027
  NDVI cutoff=0.03701, active_pixels=16384
  -> Original=109, Refined=46, Dropped=63
[REFINE] Image 1028
  NDVI cutoff=0.02773, active_pixels=16384
  -> Original=103, Refined=34, Dropped=69
[REFINE] Image 1029
  NDVI cutoff=0.03427, active_pixels=16384
  -> Original=119, Refined=56, Dropped=63
[REFINE] Image 1031
  NDVI cutoff=0.07919, active_pixels=16384
  -> Original=115, Refined=31, Dropped=84
[REFINE] Image 1032
  NDVI cutoff=0.04710, active_pixels=16384
  -> Original=116, Refined=24, Dropped=92
[REFINE] Image 1099
  NDVI cutoff=0.02238, active_pixels=16384
  -> Original=121, Refined=46, Dropped=75
[SKIP] Image 1100: GT 

[(1022, 116, 30),
 (1024, 93, 21),
 (1027, 109, 46),
 (1028, 103, 34),
 (1029, 119, 56),
 (1031, 115, 31),
 (1032, 116, 24),
 (1099, 121, 46)]

In [5]:
from src.step2_sam_run_refined import run_sam_with_refined_prompts
os.makedirs(PRED_VIS_DIR, exist_ok=True)
print("Created:", PRED_VIS_DIR)


run_sam_with_refined_prompts(dataset)


Created: d:\project_alpha\data\results\pred_vis


D:\segment-anything\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)



========== Step 2D: Running SAM with Refined Prompts ==========

Available variables in NL_1022.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1022
  -> Refined prompts=30, IoU=0.000, F1=0.000
Available variables in NL_1023.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1023
  [SKIP] Image 1023: GT empty — not evaluated.
Available variables in NL_1024.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1024
  -> Refined prompts=21, IoU=0.000, F1=0.001
Available variables in NL_1027.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1027
  -> Refined prompts=46, IoU=0.215, F1=0.353
Available variables in NL_1028.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1028
  -> Refined prompts=34, IoU=0.342, F1=0.510
Available variables in NL_1029.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']

[RUN-REFINED] Image 1029
  -> Refined prompts=56, IoU=0.000, F1=0.000
Availabl